In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import math
import xlsxwriter
from IPython.display import clear_output

In [2]:
stocks = pd.read_csv('sp500.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
498,YUM
499,ZBH
500,ZBRA
501,ZION


In [3]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [4]:
for stock in stocks['Ticker']:
    try:
        data = yf.Ticker(stock).info
        stock_dict = {
            'Ticker': [stock],
            'Stock Price': [data['currentPrice']],
            'Market Capitalization': [data['marketCap']],
            'Number of Shares to Buy': ['N/A']
        }
        df = pd.DataFrame(stock_dict)
        final_dataframe = pd.concat([final_dataframe, df], ignore_index=True)
    except:
        data = yf.Ticker(stock).info
        stock_dict = {
            'Ticker': [stock],
            'Stock Price': [data['currentPrice']],
            'Market Capitalization': [data['marketCap']],
            'Number of Shares to Buy': ['N/A']
        }
        df = pd.DataFrame(stock_dict)
        final_dataframe = pd.concat([final_dataframe, df], ignore_index=True)

    clear_output(wait=True)
    print(final_dataframe)

    Ticker  Stock Price Market Capitalization Number of Shares to Buy
0        A      118.330           34951843840                     N/A
1      AAL       18.340           11973507072                     N/A
2      AAP       69.645            4139970304                     N/A
3     AAPL      192.610         3029504884736                     N/A
4     ABBV      135.140          238426161152                     N/A
..     ...          ...                   ...                     ...
498    YUM      135.805           38037213184                     N/A
499    ZBH      141.600           29564381184                     N/A
500   ZBRA      308.000           15840532480                     N/A
501   ZION       30.970            4586687488                     N/A
502    ZTS      171.420           79215239168                     N/A

[503 rows x 4 columns]


In [5]:
portfolio_size = float(input("Enter your portfolio value: "))
portfolio_size

10000000.0

In [6]:
position_size = portfolio_size / len(final_dataframe.index)
position_size

19880.715705765408

In [7]:
for i in range(len(final_dataframe.index)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(
        position_size / final_dataframe.loc[i, 'Stock Price'])

final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,118.330,34951843840,168
1,AAL,18.340,11973507072,1084
2,AAP,69.645,4139970304,285
3,AAPL,192.610,3029504884736,103
4,ABBV,135.140,238426161152,147
...,...,...,...,...
498,YUM,135.805,38037213184,146
499,ZBH,141.600,29564381184,140
500,ZBRA,308.000,15840532480,64
501,ZION,30.970,4586687488,641


In [8]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index=False)

In [9]:
bg_color = '#0A0A23'
font_color = '#FFFFFF'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': bg_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': bg_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': bg_color,
        'border': 1
    }
)

In [10]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.close()

# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Stock Price', dollar_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', dollar_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', integer_format)

0

In [11]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalization', dollar_format],
    'D': ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

writer.close()